In [ ]:
// for local development
%install-location /notebooks/language2motion.gt/swift-install
%install-swiftpm-flags -c release
%install '.package(path: "/notebooks/language2motion.gt")' Datasets TranslationModels TextModels ModelSupport SummaryWriter LangMotionModels Checkpoints

In [ ]:
import TensorFlow
import TextModels
import TranslationModels
import Foundation
import FoundationXML
import ModelSupport
import Datasets
import SummaryWriter
import LangMotionModels
import Checkpoints
import PythonKit

In [ ]:
let np  = Python.import("numpy")

In [ ]:
%include "EnableIPythonDisplay.swift"
IPythonDisplay.shell.enable_matplotlib("inline")

In [ ]:
let device = Device.defaultTFEager

In [ ]:
let maxTextSequenceLength =  40
let maxMotionLength = 150

In [ ]:
let datasetSize: DatasetSize = .micro
let batchSize = 150

In [ ]:
let dataURL = URL(fileURLWithPath: "/notebooks/language2motion.gt/data/")
let motionDatasetURL = dataURL.appendingPathComponent("motion_dataset_v3.10Hz.\(datasetSize.rawValue)plist")

In [ ]:
/// instantiate text processor
let vocabularyURL = dataURL.appendingPathComponent("vocab.txt")
let vocabulary: Vocabulary = try! Vocabulary(fromFile: vocabularyURL)
let tokenizer: Tokenizer = BERTTokenizer(vocabulary: vocabulary, caseSensitive: false, unknownToken: "[UNK]", maxTokenLength: nil)
let textProcessor = TextProcessor(vocabulary: vocabulary, tokenizer: tokenizer)

In [ ]:
print("\nLoading dataset...")

var dataset = try Lang2Motion(
    motionDatasetURL: motionDatasetURL,
    batchSize: batchSize,
    minMotionLength: 10,
    maxMotionLength: 150,
    trainTestSplit: 1.0,
    device: device
) { (motionSample: MotionSample) -> LangMotionBatch in    
    let sentence = textProcessor.preprocess(sentence: motionSample.annotations[0], maxTextSequenceLength: maxTextSequenceLength)
    let (motionPart, target) = LangMotionBatch.preprocessTargetMotion(sampleID: motionSample.sampleID, motion: motionSample.motion, maxMotionLength: maxMotionLength)
    let source = LangMotionBatch.Source(sentence: sentence, motionPart: motionPart)
    let singleBatch = LangMotionBatch(data: source,label: target)
    return singleBatch
}

print("Dataset acquired.")

# coding

In [ ]:
// TODO: do discretization of scaled joint values
// TODO: save & load scaling info

In [ ]:
// TODO: get motion tensor

In [ ]:
// let motion = dataset.motionSamples[0].motion
// motion.shape

In [ ]:
dataset.motionSamples.count

In [ ]:
// TODO: flatten without padding

In [ ]:
let motions1 = dataset.motionSamples.map { $0.motion }

In [ ]:
let motions = Tensor(concatenating: motions1, alongAxis: 0)

In [ ]:
motions.shape

In [ ]:
//pip install feature-engine

In [ ]:
// from feature_engine.discretisers import EqualWidthDiscretiser
// discretizer = EqualFrequencyDiscretiser(q=10, variables = ['var1', 'var2'])

In [ ]:
// let discretisers  = Python.import("feature_engine.discretisers")

In [ ]:
// let discretizer = discretisers.EqualFrequencyDiscretiser(q: 10, variables: ["var1", "var2"])

In [ ]:
// discretizer

In [ ]:
let t1 = motions.flattened().expandingShape(at: 1) //motions.reshaped(to: [5907*50*47])
t1.shape

In [ ]:
let t1np = t1.makeNumpyArray()

In [ ]:
t1np.shape

In [ ]:
// discretizer.fit(t1np)

In [ ]:
// from sklearn.preprocessing import KBinsDiscretizer
let preprocessing  = Python.import("sklearn.preprocessing")

In [ ]:
// discretizer = KBinsDiscretizer(n_bins=10, encode='ordinal', strategy='quantile')
let discretizer2 = preprocessing.KBinsDiscretizer(n_bins: 200, encode: "ordinal", strategy: "quantile")
discretizer2

In [ ]:
discretizer2.fit(t1np)

In [ ]:
let t2np = discretizer2.transform(t1np)

In [ ]:
t2np[0..<10]

In [ ]:
discretizer2.inverse_transform([[40]])

## roundtrip

In [ ]:
let motion = dataset.motionSamples[0].motion
motion.shape

In [ ]:
let t3np = motion.flattened().expandingShape(at: 1).makeNumpyArray()

In [ ]:
let t4np = discretizer2.transform(t3np)
t4np.shape

In [ ]:
t4np[0..<10]

In [ ]:
let dMotion = Tensor<Int32>(Tensor<Float>(numpy: t4np)!.reshaped(like: motion))
dMotion.shape

In [ ]:
dMotion[0, 0]

In [ ]:
let dMotion_np = dMotion.flattened().expandingShape(at: 1).makeNumpyArray()
let motion_np = discretizer2.inverse_transform(dMotion_np)
motion_np.shape

In [ ]:
print(type(of:motion_np[0..<10][0][0]))

In [ ]:
Tensor<Double>(numpy: motion_np.asType)

In [ ]:
let dMotion2 = Tensor<Float>(Tensor<Double>(numpy: motion_np)!.reshaped(like: motion))
dMotion2.shape

In [ ]:
motion[0]

In [ ]:
dMotion[0]

In [ ]:
dMotion2[0]